---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [161]:
import pandas as pd
import numpy as np
doc = []
with open('./../Datasets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(5)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [298]:
# 04/20/2009; 04/20/09; 4/20/09; 4/3/09
# Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
# 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
# Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
# Feb 2009; Sep 2009; Oct 2010
# 6/2008; 12/2009
# 2009; 2010
df2 = df.str.extractall(r'(?P<Dates>(?:(?:\d{1,2})[-/](?:\d{1,2}[-/])(?:\d{2,4})|(?:\d{1,2})?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,-. ]{0,2}(?:\d{0,2}(?:nd|th|st)?)?[,-. ]{0,2}\d{2,4}|\d{4}))')

In [276]:
df2["Dates"].replace({"Jan[a-z]*":"01","Feb[a-z]*":"02","Mar[a-z]*":"03","Apr[a-z]*":"04","May[a-z]*":"05","Jun[a-z]*":"06","Jul[a-z]*":"07","Aug[a-z]*":"08","Sep[a-z]*":"09","Oct[a-z]*":"10","Nov[a-z]*":"11","Dec[a-z]*":"12","[., -]{1,2}":"/"," ":"/"},inplace = True, regex=True)

In [193]:
# df2["Dates2"] = df2[(df2["Dates"].str.contains("^\d{4}$"))].apply(lambda x : pd.to_datetime("01/01/"+x) if x.astype(int) <=2019 else pd.to_datetime("01/01/0001"))

# df2["Dates3"] = df2["Dates2"].apply(lambda x :  pd.to_datetime("01/01/"+str(int(x))) if x <=2019 else pd.to_datetime("1900/01/01"))

In [299]:
#only years yyyy
df2["Dates2"] = df2[(df2["Dates"].str.contains("^\d{4}$"))]
df2["Dates2"] = df2["Dates2"].apply(lambda x : "01/01/"+x if (pd.isnull(x) == False)&(float(x) <=2019) else np.nan)

In [300]:
# mm/dd/yy
df2["Dates3"] = 0
df2["Dates3"] = df2[df2["Dates"].str.contains("/\d{2}$")]
df2["Dates2"] = df2["Dates3"].apply(lambda x : re.sub(r'\d{2}$',"19"+str(x).rsplit("/", 1)[-1],str(x)))

In [296]:
df2["Dates2"] = df2[df2["Dates"].str.contains("^\d{1,2}/\d{1,2}/\d{4}")]

In [248]:
(df2["Dates3"].iloc[0:10])

   match
0  0        03/25/93
1  0         6/18/85
2  0          7/8/71
3  0         9/27/75
4  0          2/6/96
5  0         7/06/79
6  0         5/18/78
7  0        10/24/89
   1             NaN
8  0          3/7/86
Name: Dates3, dtype: object

In [305]:
df2[df2["Dates2"]==np.nan]

,,Dates,Dates2,Dates3
,match,,,


In [103]:
df2[df2["Dates"].str.contains("\d{0,2}/\d{2,4}$")]

,,Dates,Dates2
,match,,
0,0,03/25/93,NaN
4,0,2/6/96,NaN
7,0,10/24/89,NaN
13,0,1/26/72,NaN
15,0,1/25/2011,NaN
17,0,10/13/1976,NaN
21,0,10/21/79,NaN
23,0,2/11/76,NaN
24,0,07/25/1984,NaN


In [ ]:
df3 = df.str.extract(r'(?P<Dates>(?:(?P<Month1>\d{1,2})[-/](?P<Day1>\d{1,2}[-/])(?P<Year1>\d{2,4})|(?P<Day2>\d{1,2})?(?P<Month2>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]{0,2}(?P<Day3>\d{0,2}(?:nd|th|st)?)?[, ]{0,2}(?P<Year2>\d{2,4})|(?P<Year3>[^-]?\d{4})))')

In [ ]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here

In [269]:
x = "22/22/21"
y =re.sub(r'\d{2}$',"19"+str(x).rsplit("/", 1)[-1],x)

In [270]:
y

'22/22/1921'

In [265]:
str(x).rsplit("/", 1)[-1]

'21'